<header style="padding:1px;background:#f9f9f9;border-top:3px solid #00b2b1"><img id="Teradata-logo" src="https://www.teradata.com/Teradata/Images/Rebrand/Teradata_logo-two_color.png" alt="Teradata" width="220" align="right" />

<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Financial Services Customer Journey</b>
</header>    


<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>Introduction </b></p>

<p style = 'font-size:16px;font-family:Arial'>In this use case we will show several different analytic techniques to analyze various aspects of a customer journey using Vantage. Specifically we will use the Attribution and nPath functions.<p>

<p style = 'font-size:16px;font-family:Arial'>Here is an overview of the scenario we will be covering: </p>

<p style = 'font-size:16px;font-family:Arial'>We want to look at the important interactions customers have with a retail bank. We would like to show how we can use Vantage to discover new insights across various steps in a customer journey.</p>

<p style = 'font-size:16px;font-family:Arial'>Starting with <b>customer acquisition</b>, how do we:</p>
<ol style = 'font-size:16px;font-family:Arial'>
    <li>Find new customers</li>
    <li>Measure marketing attribution</li>
<li>How can we increase ROI and maximize marketing effectiveness, minimize the time to conversion?</li>
</ol>
<p style = 'font-size:16px;font-family:Arial'>Next we will look at <b>customer adoption</b>:</p>
<p style = 'font-size:16px;font-family:Arial'>
What leads customers to additional high end products such as wealth management accounts?
    </p>
<p style = 'font-size:16px;font-family:Arial'>Customers interact with the bank in many ways both online and offline
There are many different data sources such as in-branch interactions with tellers, online banking, email, call center logs etc. It is necessary to look at <b>all</b> of them to see the full picture.</p>

<p style = 'font-size:16px;font-family:Arial'>Vantage is very adept at aggregating data sources, it has built in connectivity to various cloud object stores as well as QueryGrid connectors directly to Hadoop, Oracle and more.
We will be using a dataset aggregated from all of these channels. Other demos cover the integration and aggregation steps, but this is out of scope in our case.</p>

<p style = 'font-size:16px;font-family:Arial'>We will see later that the insights we find are coming from these various channels both online AND offline.</p>

<hr>
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>1. Connect to Vantage</b></p>
<p style = 'font-size:16px;font-family:Arial'>You will be prompted to provide the password. Enter your password, press the Enter key, then use down arrow to go to next cell. Begin running steps with Shift + Enter keys.</p>

In [ ]:
%connect local, hidewarnings=true

<p style = 'font-size:16px;font-family:Arial'>Setup for execution of notebook. Begin running steps with Shift + Enter keys.</p>


In [ ]:
SET query_band='DEMO=FSCustomerJourney_SQL.ipynb;' UPDATE FOR SESSION;

<hr>
<b style = 'font-size:20px;font-family:Arial;color:#E37C4D'>2. Getting Data for This Demo</b>
<p style = 'font-size:16px;font-family:Arial'>We have provided data for this demo on cloud storage.  You have the option of either running the demo using foreign tables to access the data without using any storage on your environment or downloading the data to local storage which may yield somewhat faster execution, but there could be considerations of available storage.  There are two statements in the following cell, and one is commented out.  You may switch which mode you choose by changing the comment string. 


In [ ]:
call get_data('DEMO_Financial_cloud');    -- takes about 3 minutes
-- call get_data('DEMO_Financial_local');    -- takes about 7 minutes

<p style = 'font-size:16px;font-family:Arial'>Optional step – if you want to see status of databases/tables created and space used.</p>


In [ ]:
call space_report();  -- optional, takes about 10 seconds

<hr>
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>3. Customer Acquisition</b></p>

<p style = 'font-size:16px;font-family:Arial'>This is the first part of our journey. We will focus on the opening of a credit card account. We want to figure out where our customers are coming from and how can we maximize our marketing return on investment (ROI)?
We will use the powerful marketing Attribution function in Vantage to look at the multi-channel data.</p>

<p style = 'font-size:16px;font-family:Arial'>This will allow us to quantify marketing effectiveness of both our promotions and channels (online and offline). Which promotions are most effective? Using this information we can then optimize marketing spend and placement.</p>

<p style = 'font-size:16px;font-family:Arial'>We will create some tables that allow us to send large numbers of parameters programmatically to the Attribution analytic function.</p>

In [ ]:
CREATE MULTISET TABLE FSCJ_conversion_events (
    conversion_event VARCHAR(55)
)
NO PRIMARY INDEX;

<p style = 'font-size:16px;font-family:Arial'>We want to find when people have booked accounts both online and offline and use that as our success criteria: </p>

In [ ]:
INSERT INTO FSCJ_conversion_events VALUES('ACCOUNT_BOOKED_ONLINE');
INSERT INTO FSCJ_conversion_events VALUES('ACCOUNT_BOOKED_OFFLINE');

<p style = 'font-size:16px;font-family:Arial'>Vantage allows us to specify what type of attribution models we would like to apply. In this case we will keep it simple and choose a basic 'UNIFORM' strategy.</p>

In [ ]:
CREATE MULTISET TABLE FSCJ_attribution_model (
    id    INTEGER,
    model VARCHAR(100)
)
NO PRIMARY INDEX;

In [ ]:
INSERT INTO FSCJ_attribution_model VALUES(0, 'SIMPLE');
INSERT INTO FSCJ_attribution_model VALUES(1, 'UNIFORM:NA');

<p style = 'font-size:16px;font-family:Arial'>Now we are ready to invoke the Attribution function on our dataset. The dataset has all kinds of cross channel customer interactions we can analyse.</p>

In [ ]:
CREATE MULTISET TABLE FSCJ_marketing_attribution AS (
    SELECT * FROM Attribution (
        ON (
            SELECT
                customer_identifier, interaction_timestamp, 
                TRANSLATE(interaction_type USING UNICODE_TO_LATIN) AS interaction_type, customer_days_active, 
                customer_type, marketing_placement, marketing_description, marketing_category,
                interaction_type || product_category AS interaction_product
            FROM DEMO_Financial.Customer_Journey
            WHERE
                interaction_type IN ('ACCOUNT_BOOKED_OFFLINE', 'ACCOUNT_BOOKED_ONLINE', 'CLICK', 'REFERRAL', 'BROWSE')
                AND product_category <> '-1'
        ) 
        PARTITION BY customer_identifier
        ORDER BY interaction_timestamp
        ON FSCJ_conversion_events AS ConversionEventTable DIMENSION
        ON FSCJ_attribution_model AS FirstModel DIMENSION
        USING
            EventColumn ('interaction_type')
            TimestampColumn ('interaction_timestamp')
            WindowSize('rows:10')
    ) AS attrib
)
WITH DATA;

<p style = 'font-size:16px;font-family:Arial'>Marketing attribution aims to identify the events leading to the opening of a credit card account and assign value to them. The specific conversion events in the data are 'ACCOUNT_BOOKED_ONLINE', 'ACCOUNT_BOOKED_OFFLINE'. Thus calculating the most influential events and channels that are driving customer acquisition. The attribution function in Vantage supports a variety of standard attribution models. Using Vantage we can quickly see how changes to the attribution model/parameters will affect our analyses!</p>

<p style = 'font-size:16px;font-family:Arial'>Now let's get some summary statistics from the results:</p>

In [ ]:
SELECT marketing_description, AVG(attribution) AS avg_attrib, SUM(attribution) AS sum_attrib, AVG(-time_to_conversion)/3600 AS time_to_conversion
FROM FSCJ_marketing_attribution 
WHERE marketing_description NOT IN ('\N', '-1')
GROUP BY marketing_description;

In [ ]:
%chart x=avg_attrib, y=marketing_description, gridx=true, gridy=true, mark=bar, title="Average Attribution", width=700, height=300

<p style = 'font-size:16px;font-family:Arial'>The above visualization is centered around the various promotions we have run. The larger the bar the more influence they had towards a customer opening an account. </p>



In [ ]:
%chart x=sum_attrib, y=marketing_description, gridx=true, gridy=true, mark=bar, color=green, title="Total Attribution", width=700, height=300

<p style = 'font-size:16px;font-family:Arial'>The chart above shows the total attribution score the promotion had. So, the biggest drivers of signups will be the longest. Here the OTHER PROMOTIONS followed by REWARDS CARD PROMOTION and then AIRLINE CARD PROMOTION. </p>



In [ ]:
%chart x=time_to_conversion, y=marketing_description, gridx=true, gridy=true, mark=bar, color=red, title="Time to Conversion", width=700, height=300

<p style = 'font-size:16px;font-family:Arial'>The above chart shows time to conversion, so how many hours on average that promotion took place before the user went ahead and signed up. The shorter time - the faster people took action. We can see that the Gold Card Promotion II did overall the best, followed by the Hotel card and MoneySupermarket.com promotions.  </p>

<hr>
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>4. Channel analysis</b></p>

<p style = 'font-size:16px;font-family:Arial'>We have different promotions and advertising networks, now let's take a look at the traction we are getting with the various promotions across different channels:</p>

In [ ]:
SELECT marketing_category, marketing_placement, SUM(attribution) AS total_attribution 
FROM FSCJ_marketing_attribution 
WHERE marketing_description NOT IN ('\N', '-1')
GROUP BY 1, 2;

In [ ]:
%chart x=total_attribution, y=marketing_placement, gridx=true, gridy=true, mark=bar, color=purple, title="Channel Performance", width=700, height=300

<p style = 'font-size:16px;font-family:Arial'>The next visual is centered around the various channels where we ran our marketing. The length of the bar shows the overall total attribution to that channel. The colors correspond to the promotions that we looked at in the top view, so we can see which promotions were on which channels and the performance of each. In the data resultset you can see that there is email, in branch (offline) as well as web. </p>

<p style = 'font-size:16px;font-family:Arial'>For the digital channels - overall we can see that the homepage ads did the best followed by email and Google searches. </p>

<p style = 'font-size:16px;font-family:Arial'>Using a BI tool we can take this analysis to another level. We do not have BI tool available in this environment so we have captured a screen image displaying an analysis of data produced by the analysis we did using the queries above. </p>

<p style = 'font-size:16px;font-family:Arial'>The Top chart shows the Total Attributions for various kinds of promotions. The biggest drivers are the promotions that have the largest circle size. </p>
<img  src="images/ChannelPerformance.png"/>

<p style = 'font-size:16px;font-family:Arial'>The chart below with vertical lines shows the various channels and the colours of the bars indicate the type of promotion. Larger the size of the coloured bar , more the contribution of that promotion using the specified channel. </p>

<img  src="images/AttrSumm.png"/>

<p style = 'font-size:16px;font-family:Arial'>Now we can see the promotions and their success across the various channels. We can see an interesting insight that the Money Supermarket promotion and the Gold Card Promotion II were our most effective promotions. It is interesting to compare Gold Card II to the first Gold Card Promotion. Looking down at our channels we can see that the first Gold Card promotion was only run over email and the Gold Card Promotion II was run on multiple channels and it was a lot more effective for the same exact product. </p>

<p style = 'font-size:16px;font-family:Arial'>Using this dashboard and the power of Vantage can easily compare the different promotions and we can also see that we did a special promotion on MoneySupermarket.com that was particularly effective. This was only run on that particular channel and had both a quick time to conversion as well as strong average attribution.</p>

<p style = 'font-size:16px;font-family:Arial'>We can also look at some other ones. We can see that we did a Rewards Card promotion that did well across channels but was particularly effective through in branch referrals. The airline card promotion did the best on the homepage and Google vs. the other channels.</p>

<hr>
<b style = 'font-size:20px;font-family:Arial;color:#E37C4D'>5. Path To Adoption</b>

<p style = 'font-size:16px;font-family:Arial'>We want to see how our customers are opening higher end accounts such as wealth management. Many retail banks have found wealth management to be a key profit center so they are looking to build their business in this area.</p>

<p style = 'font-size:16px;font-family:Arial'>We can use powerful nPath analytic function in Vantage to do pattern/time series analysis that is very hard to do in SQL. We want to see the common paths that customers take when they go to open a wealth management account. We will also look at the affiliation between the other accounts that wealth management customers hold. </p>

<p style = 'font-size:16px;font-family:Arial'>In the code here you can see a few key points:</p>
<li style = 'font-size:16px;font-family:Arial'>We are concatenating the interaction and the product category to make unique events </li>
<li style = 'font-size:16px;font-family:Arial'>We are ignoring the starting / completing of the wealth management application because everyone does that by definition and in this case we want to reduce the noise. Further analysis could be done later on incomplete applications or other scenarios.</li>
<li style = 'font-size:16px;font-family:Arial'>The __'PATTERN'__ we are searching for is 4 events followed by opening (ACCOUNT_BOOKED) a Wealth Management account.</li>
<li style = 'font-size:16px;font-family:Arial'>The __'SYMBOLS'__ we are using is anything but opening a wealth management account is 'EVENT' and opening said account is 'ADOPTION' </li>

In [ ]:
SELECT TOP 5 *
FROM nPath (
    ON (
        SELECT
            customer_identifier,
            interaction_timestamp,
            interaction_type,
            product_category,
            interaction_type || '_' || product_category AS event,
            marketing_category,
            marketing_description,
            marketing_placement,
            sales_channel,
            conversion_sales,
            conversion_cost,
            conversion_margin
        FROM
            DEMO_Financial.Customer_Journey
        WHERE
            product_category <> '-1'
            AND interaction_type || '_' || product_category <> 'STARTS_APPLICATION_WEALTH MANAGEMENT'
            AND interaction_type || '_' || product_category <> 'COMPLETE_APPLICATION_WEALTH MANAGEMENT'
    )
    PARTITION BY
        customer_identifier
    ORDER BY
        interaction_timestamp
    USING
        MODE (NONOVERLAPPING)
        -- Limit to a depth of 4
        PATTERN ('(EVENT){4}.ADOPTION')
        SYMBOLS (
            event NOT LIKE 'ACCOUNT_BOOKED%_WEALTH MANAGEMENT' AS EVENT,
            event LIKE 'ACCOUNT_BOOKED%' AND product_category = 'WEALTH MANAGEMENT' AS ADOPTION
        )
    RESULT (
        ACCUMULATE(event OF ANY(EVENT, ADOPTION)) AS interaction_type_list,
        COUNT(event OF ANY(EVENT, ADOPTION)) AS click_depth,
        FIRST(customer_identifier OF ADOPTION) AS customer_identifier,
        FIRST(product_category OF ADOPTION) AS product_category
    )
) a;

<p style = 'font-size:18px;font-family:Arial;'>A visualization of this gives us lots of insight into the most common paths that users are taking when opening Wealth Management accounts. This is an SQL notebook and we have another Notebook for same usecase which has python code for visualizations. A Sankey Diagram was created using the output (interacion_type_list) of the nPath function used in the query above and below is the image for the nPath function.</p>

<img  src="images/PathAnalysis.png"/>

<p style = 'font-size:16px;font-family:Arial;'>We can filter it down to the most popular paths:</p>

<img  src="images/PathForTopCust.png"/>

<p style = 'font-size:16px;font-family:Arial;'>The above diagram shows paths which have at least 16 customers on the path.  Of course, this notebook is just working with small data to demonstrate the functionality. In a real-world data volume, you’d be filtering for paths repeated hundreds or thousands of time. </p>
  
<p style = 'font-size:16px;font-family:Arial;'>So let’s start by looking at the drivers of wealth management accounts online. We can see that the ‘Compare’ tool that the bank has on its’ website is a main step in opening an account. This allows you to compare its’ wealth management offerings to its competitors and this is proving to be very compelling. So, customers are browsing the offerings then using the compare tool and ultimately booking. </p>

<p style = 'font-size:16px;font-family:Arial;'>The other main drivers of online account signups are people signing up for an auto-savings plan. So, people that are savings minded tend to open wealth management accounts. Also, general browsing the offerings and opening a CD appear to be secondary paths as well. </p>

<p style = 'font-size:16px;font-family:Arial;'>Moving to offline - which we can see has different drivers. It appears to be predominately people that are opening other types of accounts as well such as CDs and Brokerage - both online and offline. We can see that the main path to booking a wealth management account offline is from in-branch referrals. So, people go into a branch and are opening another type of account and they are referred to opening a wealth management account as well!</p>

<hr>
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>6. Cleanup </b></p>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Work Tables</b></p>
<p style = 'font-size:16px;font-family:Arial'>Cleanup work tables to prevent errors next time.</p>

In [ ]:
DROP TABLE FSCJ_conversion_events;

In [ ]:
DROP TABLE FSCJ_attribution_model;

In [ ]:
DROP TABLE FSCJ_marketing_attribution;

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'> <b>Databases and Tables </b></p>
<p style = 'font-size:16px;font-family:Arial'>The following code will clean up tables and databases created above.</p>

In [ ]:
call remove_data('DEMO_Financial');-- takes about 10 seconds, optional if you want to use the data later

<hr>
<b style = 'font-size:20px;font-family:Arial;color:#E37C4D'>Dataset</b>

<p style = 'font-size:16px;font-family:Arial;'>The data for this use case is stored in the `FSCustomerJourney` database. </p>

<b style = 'font-size:20px;font-family:Arial;color:#E37C4D'>Integrated Contact History </b>

<p style = 'font-size:16px;font-family:Arial'>The data for this use case is taken from Customer_Journey table(Details below) from the DEMO_FSCustomerJourney Database. It is data from various source systems and channels already combined and put into one big table. This is all of the customer interactions, in a customer system this might be a view on top of various source tables.</p>

`Customer_Journey`

- `customer_skey`: customer key
- `customer_identifier`: unique customer identifier
- `customer_cookie`: cookie placed on customers device
- `customer_online_id`: boolean - does the customer have an online account
- `customer_offline_id`: customer account number
- `customer_type`: is this a high value customer or just a vistor browsing the website?
- `customer_days_active`: how long has the customer been active
- `interaction_session_number`: session identifier
- `interaction_timestamp`: timestamp for this event
- `interaction_source`: channel this event is from (online / offline, in branch etc.)
- `interaction_type`: type of event
- `sales_channel`: channel a sales event was in
- `conversion_id`: sales conversion identifier
- `product_category`: what type of product the event concerned (checking, savings, cd etc..)
- `product_type`: unused
- `conversion_sales`: unused
- `conversion_cost`: unused
- `conversion_margin`: unused
- `conversion_units`: unused
- `marketing_code`: marketing identifier
- `marketing_category`: marketing channel (inbranch, website, email etc..)
- `marketing_description`: marketing campaign name
- `marketing_placement`: specific marketing outlet (Google, Bloomberg.com etc..)
- `mobile_flag`: boolean was on a mobile device
- `updt`: unused

<footer style="padding:10px;background:#f9f9f9;border-bottom:3px solid #394851">©2023 Teradata. All Rights Reserved</footer>